<a href="https://colab.research.google.com/github/sg-tarek/Python/blob/main/GPT2_textgeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

In [ ]:
import pandas as pd

df = pd.read_json("https://www.dropbox.com/s/twxfxdln1l5gm3o/tweets_11-06-2020.zip?dl=1", compression='zip')
df = df[df['device'] == 'Twitter for iPhone']
df = df[df['isRetweet'] == 'f']

In [ ]:
df.head()

,id,text,isRetweet,isDeleted,device,favorites,retweets,date
1,1234653427789070300,I was thrilled to be back in the Great city of...,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50
3,1304875170860015600,The Unsolicited Mail In Ballot Scam is a major...,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58
6,1315779944002199600,"“I’m running as a proud Democrat, for the Sena...",f,f,Twitter for iPhone,149007,34897,2020-10-12 22:22:39
7,1223640662689689600,Getting a little exercise this morning! https:...,f,f,Twitter for iPhone,285863,30209,2020-02-01 16:14:02
8,1319501865625784300,https://t.co/4qwCKQOiOw,f,f,Twitter for iPhone,130822,19127,2020-10-23 04:52:14


In [ ]:
tweets = df["text"].tolist()

In [ ]:
with open("train.txt", "w") as f:
    for tweet in tweets[:-10]:
        f.writelines(tweet + "\n")

with open("test.txt", "w") as f:
    for tweet in tweets[-10:]:
        f.writelines(tweet + "\n")

In [ ]:
from simpletransformers.language_modeling import LanguageModelingModel

train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "train_batch_size": 32,
    "num_train_epochs": 5,
    "save_eval_checkpoints": False,
    "mlm": False,
    #"best_model_dir": "/content/drive/MyDrive/Colab/GPR2Trump/best" <-- after mounting google drive
}

model = LanguageModelingModel('gpt2', 'gpt2', args=train_args)

model.train_model("train.txt", eval_file="test.txt")

In [ ]:
model.eval_model("test.txt")

In [ ]:
# Saving the quantized model
model.save_model("/content/GPR2Trump", model=model.model)

# Load the saved (quantized) model
#model = ClassificationModel('roberta', "outputs/quantized", use_cuda=False) <-- load trained model with another model type

In [ ]:
import logging
from simpletransformers.language_generation import LanguageGenerationModel


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


model = LanguageGenerationModel('gpt2', "/content/GPR2Trump", args={"max_length": 100})

In [ ]:
model.generate('kill')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
INFO:simpletransformers.language_generation.language_generation_model:=== GENERATED SEQUENCE 1 ===
INFO:simpletransformers.language_generation.language_generation_model:kill the State of Nebraska for no reason whatsoever - the people of Nebraska did NOTHING for our Country. They didn’t even pay their Federal, State, and Local taxes, and it was done with the help of Crazy Nancy Pelosi and the Radical Left Democrats. They are now totally out of control. Do something!.....The Democrats must now put their money where their mouths are, and fight for their workers, their great Veterans, their families, &amp; the very special people who have


['kill the State of Nebraska for no reason whatsoever - the people of Nebraska did NOTHING for our Country. They didn’t even pay their Federal, State, and Local taxes, and it was done with the help of Crazy Nancy Pelosi and the Radical Left Democrats. They are now totally out of control. Do something!.....The Democrats must now put their money where their mouths are, and fight for their workers, their great Veterans, their families, &amp; the very special people who have']